# Debug pipeline

A pipeline that has a log statement in between each step, useful for debugging a pipeline.

In [1]:
import logging

import numpy as np
import pandas as pd

from sklego.transformers import RandomAdder
from sklego.pipeline import DebugPipeline

logging.basicConfig(
    format=('[%(funcName)s:%(lineno)d] - %(message)s'),
    level=logging.INFO
)

In [2]:
# DebugPipeline set-up
n_samples, n_features = 3, 5
X = np.random.randn(n_samples, n_features)
y = np.arange(n_samples)

steps = [
    ('random_add1', RandomAdder(noise=1)),
    ('random_add2', RandomAdder(noise=10)),
    ('random_add3', RandomAdder(noise=100)),
    ('random_add4', RandomAdder(noise=1000)),
]

print('Initial X:\n', X)

Initial X:
 [[ 0.75427933 -0.1765648   0.76545654  0.06644578  0.13356564]
 [-0.40342825  1.66331045 -1.14118871  2.1294511   2.13492511]
 [ 2.23492233 -0.04990652  0.95755557  0.49672181 -1.34275932]]


## Same as sklearn pipeline.

This pipeline behaves exactly the same as a normal pipeline.

In [3]:
pipe = DebugPipeline(steps)

pipe.fit(X, y=y)
X_out = pipe.transform(X)

print('Transformed X:\n', X_out)

Transformed X:
 [[  581.89748063  -202.99292353 -1994.46690041  1484.36584826
    198.24325579]
 [ -682.13801299  -843.94945827  -932.9329898  -1707.65176811
   -536.54733408]
 [ -123.8539221    392.98146704   772.3283302   -514.12456258
    485.99260542]]


## Log statements

Tt is possible to set a `log_callback` variable that logs in between each step.

_Note: there are __three__ log statements while there are __four__ steps, because there are __three__ moments __in between__ the steps. The output can be checked outside of the pipeline._

In [4]:
pipe = DebugPipeline(steps, log_callback='default')

pipe.fit(X, y=y)
X_out = pipe.transform(X)

print('Transformed X:\n', X_out)

[default_log_callback:30] - [RandomAdder(noise=1, random_state=None)] shape=(3, 5) time=0:00:00
[default_log_callback:30] - [RandomAdder(noise=10, random_state=None)] shape=(3, 5) time=0:00:00
[default_log_callback:30] - [RandomAdder(noise=100, random_state=None)] shape=(3, 5) time=0:00:00.000988


Transformed X:
 [[ -686.71389975   271.47560327  2002.65711584  -737.90705096
  -1753.05338191]
 [ 1236.59625233    52.44718828    62.90009499   149.36996119
   1049.66574693]
 [    4.51270124  -615.70743355   710.07706082 -1302.03714759
  -1464.05506189]]


## Set the `log_callback` function later

It is possible to set the `log_callback` later then initialisation. 

In [5]:
pipe = DebugPipeline(steps)
pipe.log_callback='default'

pipe.fit(X, y=y)
X_out = pipe.transform(X)

print('Transformed X:\n', X_out)

[default_log_callback:30] - [RandomAdder(noise=1, random_state=None)] shape=(3, 5) time=0:00:00.000501
[default_log_callback:30] - [RandomAdder(noise=10, random_state=None)] shape=(3, 5) time=0:00:00.000501
[default_log_callback:30] - [RandomAdder(noise=100, random_state=None)] shape=(3, 5) time=0:00:00.000499


Transformed X:
 [[ -374.38480872  -240.23166335  -710.89421344  1525.9636747
    477.87926037]
 [-1871.5452752    610.23963148   812.64531568  -210.11518201
    477.76414093]
 [  325.87617056  -786.16541913   449.30430885   817.06409844
   -526.57647993]]


## Custom `log_callback`

The custom log callback function expect the output of each step, which is an tuple containing the output of the step and the step itself, and the execution time of the step.

In [6]:
def log_callback(output, execution_time):
    '''My custom `log_callback` function
    
    Parameters
    ----------
    output : tuple(
            numpy.ndarray or pandas.DataFrame
            :class:estimator or :class:transformer
        )
        The output of the step and a step in the pipeline.
    execution_time : float
        The execution time of the step.
    '''
    logger = logging.getLogger(__name__)
    step_result, step = output
    logger.info(f'[{step}] shape={step_result.shape} '
                f'nbytes={step_result.nbytes} time={execution_time}')

    
pipe.log_callback = log_callback

pipe.fit(X, y=y)
X_out = pipe.transform(X)

print('Transformed X:\n', X_out)

[log_callback:16] - [RandomAdder(noise=1, random_state=None)] shape=(3, 5) nbytes=120 time=0:00:00
[log_callback:16] - [RandomAdder(noise=10, random_state=None)] shape=(3, 5) nbytes=120 time=0:00:00.001001
[log_callback:16] - [RandomAdder(noise=100, random_state=None)] shape=(3, 5) nbytes=120 time=0:00:00


Transformed X:
 [[  -38.312032   -1130.76782711  -556.23344658   927.0638022
     65.34085884]
 [-1159.7336547    466.12292756   356.87118398   464.31784642
  -1317.55240566]
 [  825.96989647  1487.68175185 -1069.94645652   907.10005847
   -341.29784009]]


## Enough logging

Remove the `log_callback` function when not needed anymore.

In [7]:
pipe.log_callback = None

pipe.fit(X, y=y)
X_out = pipe.transform(X)

print('Transformed X:\n', X_out)

Transformed X:
 [[ -775.85356537  -496.65830314  -213.41661237 -1206.46459177
    631.61716953]
 [ -625.1003683   -553.36120332  -756.82449132  -325.46883157
  -2369.63665279]
 [   91.73528849  1322.85185649  -617.08321405  1161.56476922
  -1840.6646999 ]]
